In [ ]:
#| default_exp utils

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Example Data

> The `core.NeuralForecast` class allows you to efficiently fit multiple `NeuralForecast` models for large sets of time series. It operates with pandas DataFrame `df` that identifies individual series and datestamps with the `unique_id` and `ds` columns, and the `y` column denotes the target time series variable. To assist development, we declare useful datasets that we use throughout all `NeuralForecast`'s unit tests.<br><br>

In [ ]:
#| export
import random
from itertools import chain
from typing import List, Union, Optional, Tuple
from utilsforecast.compat import DFType
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd
import torch

In [ ]:
#| hide
import matplotlib.pyplot as plt

from nbdev.showdoc import add_docs, show_doc

# 1. Synthetic Panel Data

In [ ]:
#| export
def generate_series(n_series: int,
                    freq: str = 'D',
                    min_length: int = 50,
                    max_length: int = 500,
                    n_temporal_features: int = 0,
                    n_static_features: int = 0,
                    equal_ends: bool = False,
                    seed: int = 0) -> pd.DataFrame:
    """Generate Synthetic Panel Series.

    Generates `n_series` of frequency `freq` of different lengths in the interval [`min_length`, `max_length`].
    If `n_temporal_features > 0`, then each serie gets temporal features with random values.
    If `n_static_features > 0`, then a static dataframe is returned along the temporal dataframe.
    If `equal_ends == True` then all series end at the same date.

    **Parameters:**<br>
    `n_series`: int, number of series for synthetic panel.<br>
    `min_length`: int, minimal length of synthetic panel's series.<br>
    `max_length`: int, minimal length of synthetic panel's series.<br>
    `n_temporal_features`: int, default=0, number of temporal exogenous variables for synthetic panel's series.<br>
    `n_static_features`: int, default=0, number of static exogenous variables for synthetic panel's series.<br>
    `equal_ends`: bool, if True, series finish in the same date stamp `ds`.<br>
    `freq`: str, frequency of the data, [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).<br>

    **Returns:**<br>
    `freq`: pandas.DataFrame, synthetic panel with columns [`unique_id`, `ds`, `y`] and exogenous.
    """
    seasonalities = {'D': 7, 'M': 12}
    season = seasonalities[freq]

    rng = np.random.RandomState(seed)
    series_lengths = rng.randint(min_length, max_length + 1, n_series)
    total_length = series_lengths.sum()

    dates = pd.date_range('2000-01-01', periods=max_length, freq=freq).values
    uids = [
        np.repeat(i, serie_length) for i, serie_length in enumerate(series_lengths)
    ]
    if equal_ends:
        ds = [dates[-serie_length:] for serie_length in series_lengths]
    else:
        ds = [dates[:serie_length] for serie_length in series_lengths]

    y = np.arange(total_length) % season + rng.rand(total_length) * 0.5
    temporal_df = pd.DataFrame(dict(unique_id=chain.from_iterable(uids),
                                    ds=chain.from_iterable(ds),
                                    y=y))

    random.seed(seed)
    for i in range(n_temporal_features):
        random.seed(seed)
        temporal_values = [
            [random.randint(0, 100)] * serie_length for serie_length in series_lengths
        ]
        temporal_df[f'temporal_{i}'] = np.hstack(temporal_values)
        temporal_df[f'temporal_{i}'] = temporal_df[f'temporal_{i}'].astype('category')
        if i == 0:
            temporal_df['y'] = temporal_df['y'] * \
                                  (1 + temporal_df[f'temporal_{i}'].cat.codes)

    temporal_df['unique_id'] = temporal_df['unique_id'].astype('category')
    temporal_df['unique_id'] = temporal_df['unique_id'].cat.as_ordered()

    if n_static_features > 0:
        static_features = np.random.uniform(low=0.0, high=1.0, 
                        size=(n_series, n_static_features))
        static_df = pd.DataFrame.from_records(static_features, 
                           columns = [f'static_{i}'for i in  range(n_static_features)])
        
        static_df['unique_id'] = np.arange(n_series)
        static_df['unique_id'] = static_df['unique_id'].astype('category')
        static_df['unique_id'] = static_df['unique_id'].cat.as_ordered()

        return temporal_df, static_df

    return temporal_df

In [ ]:
show_doc(generate_series, title_level=3)

In [ ]:
synthetic_panel = generate_series(n_series=2)
synthetic_panel.groupby('unique_id').head(4)

In [ ]:
temporal_df, static_df = generate_series(n_series=1000, n_static_features=2,
                                         n_temporal_features=4, equal_ends=False)
static_df.head(2)

# 2. AirPassengers Data

The classic Box & Jenkins airline data. Monthly totals of international airline passengers, 1949 to 1960.

It has been used as a reference on several forecasting libraries, since it is a series that shows clear trends and seasonalities it offers a nice opportunity to quickly showcase a model's predictions performance.

In [ ]:
#| export
AirPassengers = np.array([112., 118., 132., 129., 121., 135., 148., 148., 136., 119., 104.,
                          118., 115., 126., 141., 135., 125., 149., 170., 170., 158., 133.,
                          114., 140., 145., 150., 178., 163., 172., 178., 199., 199., 184.,
                          162., 146., 166., 171., 180., 193., 181., 183., 218., 230., 242.,
                          209., 191., 172., 194., 196., 196., 236., 235., 229., 243., 264.,
                          272., 237., 211., 180., 201., 204., 188., 235., 227., 234., 264.,
                          302., 293., 259., 229., 203., 229., 242., 233., 267., 269., 270.,
                          315., 364., 347., 312., 274., 237., 278., 284., 277., 317., 313.,
                          318., 374., 413., 405., 355., 306., 271., 306., 315., 301., 356.,
                          348., 355., 422., 465., 467., 404., 347., 305., 336., 340., 318.,
                          362., 348., 363., 435., 491., 505., 404., 359., 310., 337., 360.,
                          342., 406., 396., 420., 472., 548., 559., 463., 407., 362., 405.,
                          417., 391., 419., 461., 472., 535., 622., 606., 508., 461., 390.,
                          432.], dtype=np.float32)

In [ ]:
#| export
AirPassengersDF = pd.DataFrame({'unique_id': np.ones(len(AirPassengers)),
                                'ds': pd.date_range(start='1949-01-01',
                                                    periods=len(AirPassengers), freq=pd.offsets.MonthEnd()),
                                'y': AirPassengers})

In [ ]:
AirPassengersDF.head(12)

In [ ]:
#We are going to plot the ARIMA predictions, and the prediction intervals.
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
plot_df = AirPassengersDF.set_index('ds')

plot_df[['y']].plot(ax=ax, linewidth=2)
ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
n_static_features = 3
n_series = 5

static_features = np.random.uniform(low=0.0, high=1.0, 
                        size=(n_series, n_static_features))
static_df = pd.DataFrame.from_records(static_features, 
                   columns = [f'static_{i}'for i in  range(n_static_features)])
static_df['unique_id'] = np.arange(n_series)

In [ ]:
static_df

# 3. Panel AirPassengers Data

Extension to classic Box & Jenkins airline data. Monthly totals of international airline passengers, 1949 to 1960.

It includes two series with static, temporal and future exogenous variables, that can help to explore the performance of models like `NBEATSx` and `TFT`.

In [ ]:
#| export

# Declare Panel Data
unique_id = np.concatenate([['Airline1']*len(AirPassengers), ['Airline2']*len(AirPassengers)])
ds = np.tile(
    pd.date_range(
        start='1949-01-01', periods=len(AirPassengers), freq=pd.offsets.MonthEnd()
    ).to_numpy(), 
    2,
)
y = np.concatenate([AirPassengers, AirPassengers+300])

AirPassengersPanel = pd.DataFrame({'unique_id': unique_id, 'ds': ds, 'y': y})

# For future exogenous variables
# Declare SeasonalNaive12 and fill first 12 values with y
snaive = AirPassengersPanel.groupby('unique_id')['y'].shift(periods=12).reset_index(drop=True)
AirPassengersPanel['trend'] = range(len(AirPassengersPanel))
AirPassengersPanel['y_[lag12]'] = snaive.fillna(AirPassengersPanel['y'])

# Declare Static Data
unique_id = np.array(['Airline1', 'Airline2'])
airline1_dummy = [0, 1]
airline2_dummy = [1, 0]
AirPassengersStatic = pd.DataFrame({'unique_id': unique_id,
                                    'airline1': airline1_dummy,
                                    'airline2': airline2_dummy})

AirPassengersPanel.groupby('unique_id').tail(4)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
plot_df = AirPassengersPanel.set_index('ds')

plot_df.groupby('unique_id')['y'].plot(legend=True)
ax.set_title('AirPassengers Panel Data', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(title='unique_id', prop={'size': 15})
ax.grid()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
plot_df = AirPassengersPanel[AirPassengersPanel.unique_id=='Airline1'].set_index('ds')

plot_df[['y', 'trend', 'y_[lag12]']].plot(ax=ax, linewidth=2)
ax.set_title('Box-Cox AirPassengers Data', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

# 4. Time Features

We have developed a utility that generates normalized calendar features for use as absolute positional embeddings in Transformer-based models. These embeddings capture seasonal patterns in time series data and can be easily incorporated into the model architecture. Additionally, the features can be used as exogenous variables in other models to inform them of calendar patterns in the data.

**References**<br>
- [Haoyi Zhou, Shanghang Zhang, Jieqi Peng, Shuai Zhang, Jianxin Li, Hui Xiong, Wancai Zhang. "Informer: Beyond Efficient Transformer for Long Sequence Time-Series Forecasting"](https://arxiv.org/abs/2012.07436)<br>

In [ ]:
#| export
class TimeFeature:
    def __init__(self):
        pass

    def __call__(self, index: pd.DatetimeIndex):
        return print('Overwrite with corresponding feature')

    def __repr__(self):
        return self.__class__.__name__ + "()"

class SecondOfMinute(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.second / 59.0 - 0.5

class MinuteOfHour(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.minute / 59.0 - 0.5

class HourOfDay(TimeFeature):
    """Hour of day encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.hour / 23.0 - 0.5

class DayOfWeek(TimeFeature):
    """Hour of day encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.dayofweek / 6.0 - 0.5

class DayOfMonth(TimeFeature):
    """Day of month encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.day - 1) / 30.0 - 0.5

class DayOfYear(TimeFeature):
    """Day of year encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.dayofyear - 1) / 365.0 - 0.5

class MonthOfYear(TimeFeature):
    """Month of year encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.month - 1) / 11.0 - 0.5

class WeekOfYear(TimeFeature):
    """Week of year encoded as value between [-0.5, 0.5]"""
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.week - 1) / 52.0 - 0.5

def time_features_from_frequency_str(freq_str: str) -> List[TimeFeature]:
    """
    Returns a list of time features that will be appropriate for the given frequency string.
    Parameters
    ----------
    freq_str
        Frequency string of the form [multiple][granularity] such as "12H", "5min", "1D" etc.
    """

    if freq_str not in ['Q', 'M', 'MS', 'W', 'D', 'B', 'H', 'T', 'S']:
        raise Exception('Frequency not supported')
    
    if freq_str in ['Q','M', 'MS']:
        return [cls() for cls in [MonthOfYear]]
    elif freq_str == 'W':
        return [cls() for cls in [DayOfMonth, WeekOfYear]]
    elif freq_str in ['D','B']:
        return [cls() for cls in [DayOfWeek, DayOfMonth, DayOfYear]]
    elif freq_str == 'H':
        return [cls() for cls in [HourOfDay, DayOfWeek, DayOfMonth, DayOfYear]]
    elif freq_str == 'T':
        return [cls() for cls in [MinuteOfHour, HourOfDay, DayOfWeek, DayOfMonth, DayOfYear]]
    else:
        return [cls() for cls in [SecondOfMinute, MinuteOfHour, HourOfDay, DayOfWeek, DayOfMonth, DayOfYear]]

def augment_calendar_df(df, freq='H'):
    """
    > * Q - [month]
    > * M - [month]
    > * W - [Day of month, week of year]
    > * D - [Day of week, day of month, day of year]
    > * B - [Day of week, day of month, day of year]
    > * H - [Hour of day, day of week, day of month, day of year]
    > * T - [Minute of hour*, hour of day, day of week, day of month, day of year]
    > * S - [Second of minute, minute of hour, hour of day, day of week, day of month, day of year]
    *minute returns a number from 0-3 corresponding to the 15 minute period it falls into.
    """
    df = df.copy()

    freq_map = {
        'Q':['month'],
        'M':['month'],
        'MS':['month'],
        'W':['monthday', 'yearweek'],
        'D':['weekday','monthday','yearday'],
        'B':['weekday','monthday','yearday'],
        'H':['dayhour','weekday','monthday','yearday'],
        'T':['hourminute','dayhour','weekday','monthday','yearday'],
        'S':['minutesecond','hourminute','dayhour','weekday','monthday','yearday']
    }

    ds_col = pd.to_datetime(df.ds.values)
    ds_data = np.vstack([feat(ds_col) for feat in time_features_from_frequency_str(freq)]).transpose(1,0)
    ds_data = pd.DataFrame(ds_data, columns=freq_map[freq])
    
    return pd.concat([df, ds_data], axis=1), freq_map[freq]

In [ ]:
AirPassengerPanelCalendar, calendar_cols = augment_calendar_df(df=AirPassengersPanel, freq='M')
AirPassengerPanelCalendar.head()

In [ ]:
plot_df = AirPassengerPanelCalendar[AirPassengerPanelCalendar.unique_id=='Airline1'].set_index('ds')
plt.plot(plot_df['month'])
plt.grid()
plt.xlabel('Datestamp')
plt.ylabel('Normalized Month')
plt.show()

In [ ]:
#| export
def get_indexer_raise_missing(idx: pd.Index, vals: List[str]) -> List[int]:
    idxs = idx.get_indexer(vals)
    missing = [v for i, v in zip(idxs, vals) if i == -1]
    if missing:
        raise ValueError(f'The following values are missing from the index: {missing}')
    return idxs

# 5. Prediction Intervals

In [ ]:
#| export

class PredictionIntervals:
    """Class for storing prediction intervals metadata information."""

    def __init__(
        self,
        n_windows: int = 2,
        method: str = "conformal_distribution",
    ):
        """ 
        n_windows : int
            Number of windows to evaluate.
        method : str, default is conformal_distribution
            One of the supported methods for the computation of prediction intervals:
            conformal_error or conformal_distribution
        """
        if n_windows < 2:
            raise ValueError(
                "You need at least two windows to compute conformal intervals"
            )
        allowed_methods = ["conformal_error", "conformal_distribution"]
        if method not in allowed_methods:
            raise ValueError(f"method must be one of {allowed_methods}")
        self.n_windows = n_windows
        self.method = method

    def __repr__(self):
        return f"PredictionIntervals(n_windows={self.n_windows}, method='{self.method}')"

In [ ]:
#| export
def add_conformal_distribution_intervals(
    model_fcsts: np.array, 
    cs_df: DFType,
    model: str,
    cs_n_windows: int,
    n_series: int,
    horizon: int,
    level: Optional[List[Union[int, float]]] = None,
    quantiles: Optional[List[float]] = None,
) -> Tuple[np.array, List[str]]:
    """
    Adds conformal intervals to a `fcst_df` based on conformal scores `cs_df`.
    `level` should be already sorted. This strategy creates forecasts paths
    based on errors and calculate quantiles using those paths.
    """
    assert level is not None or quantiles is not None, "Either level or quantiles must be provided"
    
    if quantiles is None and level is not None:
        alphas = [100 - lv for lv in level]
        cuts = [alpha / 200 for alpha in reversed(alphas)]
        cuts.extend(1 - alpha / 200 for alpha in alphas)
    elif quantiles is not None:
        cuts = quantiles
    
    scores = cs_df[model].to_numpy().reshape(n_series, cs_n_windows, horizon)
    scores = scores.transpose(1, 0, 2)
    # restrict scores to horizon
    scores = scores[:,:,:horizon]
    mean = model_fcsts.reshape(1, n_series, -1)
    scores = np.vstack([mean - scores, mean + scores])
    scores_quantiles = np.quantile(
        scores,
        cuts,
        axis=0,
    )
    scores_quantiles = scores_quantiles.reshape(len(cuts), -1).T
    if quantiles is None and level is not None:
        lo_cols = [f"{model}-lo-{lv}" for lv in reversed(level)]
        hi_cols = [f"{model}-hi-{lv}" for lv in level]
        out_cols = lo_cols + hi_cols
    elif quantiles is not None:
        out_cols = [f"{model}-ql{q}" for q in quantiles]

    fcsts_with_intervals = np.hstack([model_fcsts, scores_quantiles])

    return fcsts_with_intervals, out_cols

In [ ]:
#| export
def add_conformal_error_intervals(
    model_fcsts: np.array, 
    cs_df: DFType, 
    model: str,
    cs_n_windows: int,
    n_series: int,
    horizon: int,
    level: Optional[List[Union[int, float]]] = None,
    quantiles: Optional[List[float]] = None,
) -> Tuple[np.array, List[str]]:
    """
    Adds conformal intervals to a `fcst_df` based on conformal scores `cs_df`.
    `level` should be already sorted. This startegy creates prediction intervals
    based on the absolute errors.
    """
    assert level is not None or quantiles is not None, "Either level or quantiles must be provided"

    if quantiles is None and level is not None:
        alphas = [100 - lv for lv in level]
        cuts = [alpha / 200 for alpha in reversed(alphas)]
        cuts.extend(1 - alpha / 200 for alpha in alphas)
    elif quantiles is not None:
        cuts = quantiles

    mean = model_fcsts.ravel()
    scores = cs_df[model].to_numpy().reshape(n_series, cs_n_windows, horizon)
    scores = scores.transpose(1, 0, 2)
    # restrict scores to horizon
    scores = scores[:,:,:horizon]
    scores_quantiles = np.quantile(
        scores,
        cuts,
        axis=0,
    )
    scores_quantiles = scores_quantiles.reshape(len(cuts), -1)          

    if quantiles is None and level is not None:
        lo_cols = [f"{model}-lo-{lv}" for lv in reversed(level)]
        hi_cols = [f"{model}-hi-{lv}" for lv in level]
        out_cols = lo_cols + hi_cols
    else:
        out_cols = [f"{model}-ql{q}" for q in cuts]
    
    scores_quantiles_ls = []
    for i, q in enumerate(cuts):
        if q < 0.5:
            scores_quantiles_ls.append(mean - scores_quantiles[::-1][i])
        elif q > 0.5:
            scores_quantiles_ls.append(mean + scores_quantiles[i])
        else:
            scores_quantiles_ls.append(mean)
    scores_quantiles = np.vstack(scores_quantiles_ls).T    

    fcsts_with_intervals = np.hstack([model_fcsts, scores_quantiles])

    return fcsts_with_intervals, out_cols

In [ ]:
#| export
def get_prediction_interval_method(method: str):
    available_methods = {
        "conformal_distribution": add_conformal_distribution_intervals,
        "conformal_error": add_conformal_error_intervals,
    }
    if method not in available_methods.keys():
        raise ValueError(
            f"prediction intervals method {method} not supported "
            f'please choose one of {", ".join(available_methods.keys())}'
        )
    return available_methods[method]

In [ ]:
#| export
def level_to_quantiles(level: List[Union[int, float]]) -> List[float]:
    """
    Converts a list of levels to a list of quantiles.
    """
    level_set = set(level)
    return sorted(list(set(sum([[(50 - l / 2) / 100, (50 + l / 2) / 100] for l in level_set], []))))

def quantiles_to_level(quantiles: List[float]) -> List[Union[int, float]]:
    """
    Converts a list of quantiles to a list of levels.
    """
    quantiles_set = set(quantiles)
    return sorted(set([int(round(100 - 200 * (q * (q < 0.5) + (1 - q) * (q >= 0.5)), 2)) for q in quantiles_set]))

In [ ]:
#| hide
# Test level_to_quantiles
level_base = [80, 90]
quantiles_base = [0.05, 0.1, 0.9, 0.95]
quantiles = level_to_quantiles(level_base)
level = quantiles_to_level(quantiles_base)

assert quantiles == quantiles_base
assert level == level_base

# 6. Explainability

In [ ]:
#| export

# class ShapModelWrapper(torch.nn.Module):
#     """
#     SHAP wrapper model that converts flattened tensor input to dictionary format
#     and handles multiple series dynamically.
#     """
#     def __init__(self, original_model, train_df, static_df):
#         super().__init__()
#         self.original_model = original_model
#         self.train_df = train_df
#         self.static_df = static_df
#         self.futr_exog_cols = original_model.futr_exog_list
#         self.hist_exog_cols = original_model.hist_exog_list
#         self.stat_exog_cols = original_model.stat_exog_list
#         self.input_size = original_model.input_size
#         self.h = original_model.h
        
#         # Calculate input dimensions
#         self.n_futr_features = len(self.futr_exog_cols) * (self.input_size + self.h) if self.futr_exog_cols else 0
#         self.n_hist_exog_features = len(self.hist_exog_cols) * self.input_size if self.hist_exog_cols else 0
#         self.n_hist_target_features = self.input_size
#         self.n_series_features = 1  # unique_id encoded as integer
        
#         # Create mapping for unique_ids to static features (only if static features exist)
#         self.static_mapping = {}
#         if self.stat_exog_cols and static_df is not None:
#             for unique_id in static_df['unique_id'].unique():
#                 static_values = static_df[static_df['unique_id'] == unique_id][self.stat_exog_cols].values[0]
#                 self.static_mapping[unique_id] = torch.tensor(static_values, dtype=torch.float32)
        
#         # Create unique_id to integer mapping using training data
#         available_unique_ids = sorted(self.train_df['unique_id'].unique())
#         self.unique_id_to_int = {uid: i for i, uid in enumerate(available_unique_ids)}
#         self.int_to_unique_id = {i: uid for uid, i in self.unique_id_to_int.items()}

#     def forward(self, X_flat):
#         """
#         Convert flattened tensor input to dictionary format and call original model
#         X_flat: [batch_size, n_total_features] where features include series_id + futr_exog + hist_exog + hist_target
#         """
#         batch_size = X_flat.shape[0]
        
#         # Split the input tensor
#         idx = 0
        
#         # Series identifier (first feature)
#         series_ids = X_flat[:, idx:idx + self.n_series_features].long().flatten()
#         idx += self.n_series_features
        
#         # Future exogenous features (varying or None)
#         if self.futr_exog_cols:
#             futr_flat = X_flat[:, idx:idx + self.n_futr_features]
#             idx += self.n_futr_features
#             futr_exog = futr_flat.reshape(batch_size, self.input_size + self.h, len(self.futr_exog_cols))
#         else:
#             futr_exog = None
        
#         # Historical exogenous features (varying or None)
#         if self.hist_exog_cols:
#             hist_exog_flat = X_flat[:, idx:idx + self.n_hist_exog_features]
#             hist_exog = hist_exog_flat.reshape(batch_size, self.input_size, len(self.hist_exog_cols))
#             idx += self.n_hist_exog_features
#         else:
#             hist_exog = None
        
#         # Historical target values (always present)
#         hist_target_flat = X_flat[:, idx:idx + self.n_hist_target_features]
#         insample_y = hist_target_flat.reshape(batch_size, self.input_size)
        
#         # Static exogenous features (varies by series, None if no static features)
#         if self.stat_exog_cols and self.static_mapping:
#             stat_exog_batch = []
#             for i in range(batch_size):
#                 series_int = series_ids[i].item()
#                 unique_id = self.int_to_unique_id[series_int]
#                 stat_exog_batch.append(self.static_mapping[unique_id])
#             stat_exog = torch.stack(stat_exog_batch)
#         else:
#             stat_exog = None
        
#         # Create windows_batch dictionary
#         windows_batch = {
#             'insample_y': insample_y.unsqueeze(-1),
#             'futr_exog': futr_exog,
#             'hist_exog': hist_exog,
#             'stat_exog': stat_exog,
#             'insample_mask': torch.ones(batch_size, self.input_size, dtype=torch.bool)
#         }
        
#         # Call original model
#         return self.original_model(windows_batch)

class ShapModelWrapper:
    """
    SHAP wrapper that uses nf.predict() to properly handle scaling
    """
    def __init__(self, nf_object, model, train_df, static_df, futr_df=None):
        self.nf = nf_object  # Already fitted NeuralForecast object
        self.model = model
        self.train_df = train_df
        self.static_df = static_df
        self.futr_df = futr_df
        self.freq = nf_object.freq
        
        self.futr_exog_cols = model.futr_exog_list
        self.hist_exog_cols = model.hist_exog_list
        self.stat_exog_cols = model.stat_exog_list
        self.input_size = model.input_size
        self.h = model.h
        self.model_alias = model.alias or type(model).__name__
        self.model.trainer_kwargs['logger'] = False
        
        # Calculate input dimensions
        self.n_futr_features = len(self.futr_exog_cols) * (self.input_size + self.h) if self.futr_exog_cols else 0
        self.n_hist_exog_features = len(self.hist_exog_cols) * self.input_size if self.hist_exog_cols else 0
        self.n_hist_target_features = self.input_size
        self.n_series_features = 1  # unique_id encoded as integer
        
        # Create unique_id to integer mapping
        available_unique_ids = sorted(self.train_df['unique_id'].unique())
        self.unique_id_to_int = {uid: i for i, uid in enumerate(available_unique_ids)}
        self.int_to_unique_id = {i: uid for uid, i in self.unique_id_to_int.items()}
        
        # Store static features mapping if needed
        self.static_mapping = {}
        if self.stat_exog_cols and static_df is not None:
            for unique_id in static_df['unique_id'].unique():
                static_values = static_df[static_df['unique_id'] == unique_id][self.stat_exog_cols].values[0]
                self.static_mapping[unique_id] = static_values

    def _predict_single_sample(self, x_flat_single, horizon_idx=None):
        """Process a single sample - extracted from predict_batch for parallelization"""
        # Parse the flattened input
        idx = 0
        
        # Series identifier
        series_int = int(x_flat_single[idx])
        unique_id = self.int_to_unique_id[series_int]
        idx += self.n_series_features
        
        # Future exogenous features (if present)
        temp_futr_df = None
        if self.futr_exog_cols and self.futr_df is not None:
            futr_flat = x_flat_single[idx:idx + self.n_futr_features]
            idx += self.n_futr_features
            
            # Reshape to (input_size + h, n_futr_features)
            futr_reshaped = futr_flat.reshape(self.input_size + self.h, len(self.futr_exog_cols))
            
            # Get the future part only (last h timesteps)
            futr_future = futr_reshaped[-self.h:, :]
            
            # Create future DataFrame with dummy consecutive dates
            series_train_data = self.train_df[self.train_df['unique_id'] == unique_id]
            last_train_date = series_train_data['ds'].iloc[-1]

            if pd.api.types.is_datetime64_any_dtype(series_train_data['ds']):
                future_dates = pd.date_range(start=last_train_date, periods=self.h + 1, freq=self.freq)[1:]
            else:
                future_dates = np.arange(last_train_date + 1, last_train_date + self.h + 1)
            
            temp_futr_df = pd.DataFrame(futr_future, columns=self.futr_exog_cols)
            temp_futr_df['ds'] = future_dates[:self.h]
            temp_futr_df['unique_id'] = unique_id
        
        # Create a copy of ALL training data
        temp_train_df = self.train_df.copy()
        series_mask = temp_train_df['unique_id'] == unique_id
        series_indices = temp_train_df[series_mask].index
        
        # Historical exogenous features (if present)
        if self.hist_exog_cols:
            hist_exog_flat = x_flat_single[idx:idx + self.n_hist_exog_features]
            hist_exog_reshaped = hist_exog_flat.reshape(self.input_size, len(self.hist_exog_cols))
            idx += self.n_hist_exog_features
            
            last_indices = series_indices[-self.input_size:]
            for i, col in enumerate(self.hist_exog_cols):
                temp_train_df.loc[last_indices, col] = hist_exog_reshaped[:, i]
        
        # Historical target values
        hist_target = x_flat_single[idx:idx + self.n_hist_target_features]
        
        last_indices = series_indices[-self.input_size:]
        temp_train_df.loc[last_indices, 'y'] = hist_target
        
        # Prepare futr_df for ALL series
        if temp_futr_df is not None:
            all_futr_df = []
            for uid in temp_train_df['unique_id'].unique():
                if uid == unique_id:
                    all_futr_df.append(temp_futr_df)
                else:
                    if self.futr_df is not None:
                        other_series_futr = self.futr_df[self.futr_df['unique_id'] == uid].copy()
                        if len(other_series_futr) > 0:
                            other_series_futr = other_series_futr.iloc[:self.h]
                            all_futr_df.append(other_series_futr)
                        else:
                            other_train_data = temp_train_df[temp_train_df['unique_id'] == uid]
                            last_date = other_train_data['ds'].iloc[-1]

                            if pd.api.types.is_datetime64_any_dtype(other_train_data['ds']):
                                future_dates = pd.date_range(start=last_date, periods=self.h + 1, freq=self.freq)[1:]
                            else:
                                future_dates = np.arange(last_date + 1, last_date + self.h + 1)
                            
                            dummy_futr = pd.DataFrame()
                            for col in self.futr_exog_cols:
                                dummy_futr[col] = np.zeros(self.h)
                            dummy_futr['ds'] = future_dates[:self.h]
                            dummy_futr['unique_id'] = uid
                            all_futr_df.append(dummy_futr)
            
            combined_futr_df = pd.concat(all_futr_df, ignore_index=True) if all_futr_df else None
        else:
            combined_futr_df = None
        
        # Use the already fitted NeuralForecast object to predict
        forecast = self.nf.predict(df=temp_train_df, static_df=self.static_df, futr_df=combined_futr_df)
        
        # Extract predictions for this series
        series_forecast = forecast[forecast['unique_id'] == unique_id].reset_index(drop=True)
        
        if len(series_forecast) == 0:
            raise ValueError(f"No predictions found for series {unique_id}")
        
        if self.model_alias not in series_forecast.columns:
            model_cols = [col for col in series_forecast.columns if col not in ['unique_id', 'ds']]
            if model_cols:
                actual_model_col = model_cols[0]
            else:
                raise ValueError(f"Model column {self.model_alias} not found in forecast columns: {series_forecast.columns.tolist()}")
        else:
            actual_model_col = self.model_alias
        
        if horizon_idx is not None:
            if horizon_idx >= len(series_forecast):
                raise IndexError(f"horizon_idx {horizon_idx} is out of bounds. Only {len(series_forecast)} horizons available.")
            pred = series_forecast[actual_model_col].iloc[horizon_idx]
        else:
            pred = series_forecast[actual_model_col].mean()
        
        return pred


    def predict_batch(self, X_flat, horizon_idx=None):
        """
        Convert flattened input to DataFrames and use nf.predict()
        Now with parallel processing for better performance
        """
        batch_size = X_flat.shape[0]
        
        # Use ThreadPoolExecutor for parallel processing
        # Adjust max_workers based on your system (2-8 typically works well)
        max_workers = min(4, batch_size)  # Don't create more threads than samples
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            futures = [
                executor.submit(self._predict_single_sample, X_flat[b], horizon_idx)
                for b in range(batch_size)
            ]
            
            # Collect results in order
            predictions = [future.result() for future in futures]
        
        return np.array(predictions)

In [ ]:
#| export

def create_input_tensor_for_series(train_df, unique_id, wrapper_model, futr_df=None):
    """Create input tensor for a specific series"""
    train_series = train_df[train_df['unique_id'] == unique_id]
    
    input_components = []
    futr_exog_cols = wrapper_model.futr_exog_cols
    hist_exog_cols = wrapper_model.hist_exog_cols
    input_size = wrapper_model.input_size
    
    # Series identifier (encoded as integer)
    series_int = wrapper_model.unique_id_to_int[unique_id]
    input_components.append(np.array([series_int]))
    
    # Future exogenous features (if they exist) - FULL LENGTH (historical + future)
    if futr_exog_cols:
        if futr_df is None:
            raise ValueError("You must pass a futr_df if futr_exog_list is specified.")
        futr_exog_series = futr_df[futr_df['unique_id'] == unique_id].reset_index(drop=True)
        # Get historical part
        futr_hist_data = train_series[futr_exog_cols].values[-input_size:]
        # Get future part  
        futr_pred_data = futr_exog_series[futr_exog_cols].values
        # Combine and flatten
        full_futr_data = np.vstack([futr_hist_data, futr_pred_data]).flatten()
        input_components.append(full_futr_data)
    
    # Historical exogenous features (if they exist)
    if hist_exog_cols:
        hist_exog_data = train_series[hist_exog_cols].values[-input_size:].flatten()
        input_components.append(hist_exog_data)
    
    # Historical target values (always present)
    hist_target_data = train_series['y'].values[-input_size:]
    input_components.append(hist_target_data)
    
    # Combine all features
    complete_input = np.concatenate(input_components)
    return complete_input.reshape(1, -1)

In [ ]:
#| export

def create_multi_series_background_data(train_df, wrapper_model, n_samples_per_series=10):
    """Create background data including samples from all series"""
    background_samples = []
    
    futr_exog_cols = wrapper_model.futr_exog_cols
    hist_exog_cols = wrapper_model.hist_exog_cols
    input_size = wrapper_model.input_size
    horizon = wrapper_model.h
    
    for unique_id in train_df['unique_id'].unique():
        train_series = train_df[train_df['unique_id'] == unique_id]
        
        # Determine the range of valid indices
        start_idx = input_size
        if futr_exog_cols:
            end_idx = len(train_series) - horizon + 1
        else:
            end_idx = len(train_series)
        
        # Sample points from this series
        if end_idx > start_idx:
            sample_indices = np.linspace(start_idx, end_idx - 1, 
                                       min(n_samples_per_series, end_idx - start_idx), 
                                       dtype=int)
            
            for i in sample_indices:
                sample_components = []
                
                # Series identifier
                series_int = wrapper_model.unique_id_to_int[unique_id]
                sample_components.append(np.array([series_int]))
                
                # Future exogenous features (if they exist) - FULL LENGTH
                if futr_exog_cols:
                    futr_hist_data = train_series[futr_exog_cols].iloc[i-input_size:i].to_numpy().flatten()
                    futr_pred_data = train_series[futr_exog_cols].iloc[i:i + horizon].to_numpy().flatten()
                    full_futr_data = np.concatenate([futr_hist_data, futr_pred_data])
                    sample_components.append(full_futr_data)
                
                # Historical exogenous features (if they exist)
                if hist_exog_cols:
                    hist_exog_window = train_series[hist_exog_cols].iloc[i-input_size:i].to_numpy().flatten()
                    sample_components.append(hist_exog_window)
                
                # Historical target values
                hist_target_window = train_series['y'].iloc[i-input_size:i].to_numpy()
                sample_components.append(hist_target_window)
                
                # Combine all features
                complete_sample = np.concatenate(sample_components)
                background_samples.append(complete_sample)
    
    return np.array(background_samples)

In [ ]:
#| export

def model_predict(X, wrapper_model):
    """Prediction function that takes numpy array and returns predictions"""
    X_tensor = torch.tensor(X, dtype=torch.float32)
    with torch.no_grad():
        predictions = wrapper_model(X_tensor)
    return predictions.numpy()

In [ ]:
#| export

def create_multi_series_feature_names(wrapper_model):
    """Create feature names for multi-series input"""
    feature_names = ['series_id']
    futr_exog_cols = wrapper_model.futr_exog_cols
    hist_exog_cols = wrapper_model.hist_exog_cols
    input_size = wrapper_model.input_size
    horizon = wrapper_model.h
    
    # Future exogenous features (if they exist) - FULL LENGTH
    if futr_exog_cols:
        for i in range(input_size):
            for col in futr_exog_cols:
                feature_names.append(f'{col}_hist_lag{i+1}')
        for i in range(horizon):
            for col in futr_exog_cols:
                feature_names.append(f'{col}_h{i+1}')
    
    # Historical exogenous features (if they exist)
    if hist_exog_cols:
        for i in range(input_size):
            for col in hist_exog_cols:
                feature_names.append(f'{col}_lag{i+1}')
    
    # Historical target values (always present)
    for i in range(input_size):
        feature_names.append(f'y_lag{i+1}')
    
    return feature_names

# ======================= NOT USED ========================================== #

def group_features_data(X, feature_groups):
    """
    Group features by averaging within each group
    X: [n_samples, n_features] including series_id
    feature_groups: list of arrays with feature indices
    Returns: [n_samples, n_groups + 1] where +1 is for series_id
    """
    n_samples = X.shape[0]
    n_groups = len(feature_groups) + 1  # +1 for series_id
    # Use float32 for consistency
    X_grouped = np.zeros((n_samples, n_groups), dtype=np.float32)
    
    # Keep series_id as is
    X_grouped[:, 0] = X[:, 0].astype(np.float32)
    
    # Group other features by averaging
    for group_idx, feature_indices in enumerate(feature_groups):
        X_grouped[:, group_idx + 1] = X[:, feature_indices].mean(axis=1).astype(np.float32)
    
    return X_grouped

def expand_grouped_features(X_grouped, feature_groups, original_dim):
    """
    Expand grouped features back to original dimension
    X_grouped: [n_samples, n_groups] including series_id
    Returns: [n_samples, original_dim]
    """
    n_samples = X_grouped.shape[0]
    # Use float32 to match typical PyTorch/neural network dtypes
    X_expanded = np.zeros((n_samples, original_dim), dtype=np.float32)
    
    # Copy series_id
    X_expanded[:, 0] = X_grouped[:, 0].astype(np.float32)
    
    # Expand each group by replicating the averaged value
    for group_idx, feature_indices in enumerate(feature_groups[1:]):  # Skip series_id group
        # Replicate the grouped value to all features in the group
        # Cast to float32 to avoid dtype warnings
        X_expanded[:, feature_indices] = X_grouped[:, group_idx + 1:group_idx + 2].astype(np.float32)
    
    return X_expanded

def create_feature_groups(wrapper_model):
    """Create feature groups for SHAP"""
    groups = []
    idx = 0
    
    # Group 0: Series ID (we'll exclude this later)
    groups.append(np.array([idx]))
    idx += 1
    
    # Group 1: Future exogenous (if exists)
    futr_group = []
    if wrapper_model.futr_exog_cols:
        n_futr = wrapper_model.n_futr_features
        futr_group = np.arange(idx, idx + n_futr)
        groups.append(futr_group)
        idx += n_futr
    
    # Group 2: Historical exogenous (if exists)
    hist_exog_group = []
    if wrapper_model.hist_exog_cols:
        n_hist_exog = wrapper_model.n_hist_exog_features
        hist_exog_group = np.arange(idx, idx + n_hist_exog)
        groups.append(hist_exog_group)
        idx += n_hist_exog
    
    # Group 3: Historical target (lags)
    lags_group = np.arange(idx, idx + wrapper_model.n_hist_target_features)
    groups.append(lags_group)
    
    # Create group names
    group_names = ['series_id']
    if wrapper_model.futr_exog_cols:
        group_names.append('futr_exog')
    if wrapper_model.hist_exog_cols:
        group_names.append('hist_exog')
    group_names.append('lags')
    
    return groups, group_names